In [19]:
from functions import *

# Usage
folder_path = 'Input'  # Update with the correct folder path
output_path = 'Output'  # Output PNG file path

create_booklet(folder_path, output_path)

C:\Users\Olivi\AppData\Local\Temp\ipykernel_24492\2620811856.py:93: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  a6_image = Image.open(input_paths[page_num]).resize(a6_size, Image.ANTIALIAS)
